In [1]:
%matplotlib inline

import sys
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
sys.path.append("/Users/arankhanna/Dropbox/Dolores/Iverson")

from IPython.display import display, display_javascript
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.display import Javascript
from IPython.core.display import display, HTML
from ipywidgets import IntSlider, Text, VBox
# This shit should always run for dashboards
display(HTML("<style>.container { width:100% !important; }</style>"))

# State for Dashboard
done = False
wizards = []

from pipelines.io.markets.daily_marketcap_tracker import DailyMarketcapTracker
from pipelines.io.code.git_tracker import GitTracker
from pipelines.io.social.telegram_tracker import TelegramTracker
from pipelines.io.markets.market_tracker import MarketTracker
from pipelines.io.markets.orderbook_tracker import OrderbookTracker

# Base pipelines
dmt = DailyMarketcapTracker()
gt = GitTracker() 
tt = TelegramTracker() 
mt = MarketTracker()
ot = OrderbookTracker(mt)

# Hi I'm a widget message
display(HTML("<h1>Welcome to the IV Lab Dashboard!</h1>\
<h2>This area titled 'Hidden Cells' is where the available Wizards outside of your workspace live<h2/>\
<h3>To move these up into your workspace hover over and click the button in the top right</h3>\
<h4>And to move widgets down from your workspace above hover over and click X the button in the top right</h4>"))

In [2]:
# Start registry here
from wizards.crypto_returns_wizard import DailyReturnsWizard
if done:
    wizards.append(DailyReturnsWizard(dmt))

In [3]:
from wizards.crypto_daily_wizard import CandlestickWizard
if done:
    wizards.append(CandlestickWizard(dmt))

In [4]:
from wizards.git_wizard import CodeWizard
if done:
    wizards.append(CodeWizard(gt))

In [5]:
from wizards.telegram_chat_wizard import ChatWizard
if done:
    wizards.append(ChatWizard(tt))

In [6]:
from wizards.crypto_listing_wizard import MarketListingWizard

if done:
    wizards.append(MarketListingWizard(mt))

In [7]:
from wizards.crypto_orderbook_wizard import OrderbookWizard

if done:
    wizards.append(OrderbookWizard(ot))

In [8]:
''' 
xxxxxxxxxxxxxx
xxxxxxxxxxxxxx
WIDGET WIDGET 
xxxxxxxxxxxxxx
xxxxxxxxxxxxxx
'''

this_cell = 9 # This should be the last UI cell so use this ID specifically
widget_map = {
    'Returns Analysis':1,
    'Price Charts':2,
    'Code Explorer':3,
    'Chat Explorer':4,
    'Market Listing':5,
    'Orderbook Explorer':6,
}


widget_types = widgets.Select(
    options=widget_map.keys(),
    description='Wizards',
    disabled=False
)

def add_wizard(x):
    widget_id = widget_map[widget_types.value]
    
    # JS to add and execute a cell w/ certain ID
    js = '''
    var cell_id = %s;
    var all_cells =Jupyter.notebook.get_cells();
    var cells = [all_cells[cell_id]]
    Jupyter.notebook.clipboard = [];
    var cell_json;
    for (var i=0; i < cells.length; i++) {
        cell_json = cells[i].toJSON();
        if (cell_json.metadata.deletable !== undefined) {
            delete cell_json.metadata.deletable;
        }
        Jupyter.notebook.clipboard.push(cell_json);
    }
    Jupyter.notebook.enable_paste();

    // Paste
    if (Jupyter.notebook.clipboard !== null && Jupyter.notebook.paste_enabled) {
        var first_inserted = null;
        for (var i = Jupyter.notebook.clipboard.length-1; i >= 0; i--) {
            var cell_data = Jupyter.notebook.clipboard[i];
            var new_cell = Jupyter.notebook.insert_cell_below(cell_data.cell_type);
            new_cell.fromJSON(cell_data);
            if (first_inserted === null) {
                first_inserted = new_cell;
            }
        }
        first_inserted.focus_cell();
    }

    // Execute
    Jupyter.notebook.execute_cell();
    $('button[data-dashboard-state="notebook"]').click();
    $('button[data-dashboard-state="grid"]').click();   
    '''%(str(widget_id))
    
    display_javascript(Javascript(js))
        
# def remove_wizards(x):
#     global widget_ids
#     id_list = [i[1] for i in widget_ids]
    
#     # JS to add and execute a cell w/ certain ID
#     js = '''Jupyter.notebook.delete_cells(%s)'''%(str(id_list))
    
#     display_javascript(Javascript(js))
#     widget_ids = []

    
add_widget_button = widgets.Button(description="Create New")
add_widget_button.on_click(add_wizard)
# remove_widget_button = widgets.Button(description="Clear Wizards")
# remove_widget_button.on_click(remove_wizards)

display(widget_types)
display(add_widget_button)
# display(remove_widget_button)

Select(description=u'Wizards', options=('Market Listing', 'Chat Explorer', 'Returns Analysis', 'Orderbook Explorer', 'Code Explorer', 'Price Charts'), value='Market Listing')

Button(description=u'Create New', style=ButtonStyle())

In [9]:
done = True
js = '''$('button[data-dashboard-state="grid"]').click();'''
display_javascript(Javascript(js))